# 1. Mount virtual drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Importing libraries

In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from pandas import ExcelWriter
from sklearn.metrics import accuracy_score, recall_score, auc, roc_curve, precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# 3. Loading datasets

In [40]:
train_df = pd.read_csv("/content/drive/My Drive/train.csv",encoding = 'ISO-8859-1')
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")
address_df = pd.read_csv("/content/drive/MyDrive/addresses.csv")

train_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,60606,NaN,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,48208,NaN,USA,2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,48223,NaN,USA,2004-04-26 13:40:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,48214,NaN,USA,2004-04-26 13:30:00,NaN,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,48206,NaN,USA,2004-04-26 13:00:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [4]:
train_df.shape

(250306, 34)

In [7]:
test_df.head()

,ticket_id,violation_street_number,violation_street_name,mailing_address_str_name,city,ticket_issued_date,hearing_date,violation_description,disposition,fine_amount,late_fee,judgment_amount
0,284932,10041.0,ROSEBERRY,ROSEBERRY,DETROIT,2012-01-04 14:00:00,2012-01-19 09:00:00,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,250.0
1,285362,18520.0,EVERGREEN,GLASTONBURY,DETROIT,2012-01-05 09:50:00,2012-02-06 09:00:00,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,100.0,1130.0
2,285361,18520.0,EVERGREEN,GLASTONBURY,DETROIT,2012-01-05 09:50:00,2012-02-06 09:00:00,Improper placement of Courville container betw...,Responsible by Default,100.0,10.0,140.0
3,285338,1835.0,CENTRAL,NELSON,WOODHAVEN,2012-01-05 10:25:00,2012-02-07 09:00:00,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,250.0
4,285346,1700.0,CENTRAL,MUNGER,LIVONIA,2012-01-05 10:20:00,2012-02-14 09:00:00,Violation of time limit for approved container...,Responsible by Default,100.0,10.0,140.0


In [8]:
test_df.shape

(61001, 12)

# 4. Feature engineering

### 4.1 Data cleaning

In [ ]:
#Changing all city entries to upper case to make it simpler
train_df["city"] = train_df["city"].apply(str.upper)

#Filtering tickets only for the city of Detroit and then dropping the 'city' variable 
#since all instances would be for Detroit
train_df = train_df[train_df["city"]=="DETROIT"]
train_df = train_df.drop(["city"],axis=1)
test_df = test_df.drop(["city"],axis=1)

#Removing all 'not found responsible' compliance entries
train_df = train_df[(train_df["compliance"]==1.0) | (train_df["compliance"]==0.0)]

#Dropping all the 'Nan' instances from the training set
train_df = train_df.dropna()

#Removing original time variables
train_df = train_df.drop(["ticket_issued_date","hearing_date"],axis=1)
test_df = test_df.drop(["ticket_issued_date","hearing_date"],axis=1)

### 4.2 Feature engineering

In [ ]:
#Turning the late_fee variable to binary values with 0 indicating no late fee and 1 indicating a late fee
train_df["late_fee_bin"] = (train_df["late_fee"]>0).map(int)
train_df = train_df.drop(["late_fee"],axis=1)
test_df["late_fee_bin"] = (test_df["late_fee"]>0).map(int)
test_df = test_df.drop(["late_fee"],axis=1)

#Creating a new variable for storing the time difference between the ticket issued date and hearing date
#And converting the difference into days for easier calculation
train_df["ticket_issued_date"] = train_df.apply(lambda x: pd.to_datetime(x[6]),axis=1)
train_df["hearing_date"] = train_df.apply(lambda x: pd.to_datetime(x[7]),axis=1)
test_df["ticket_issued_date"] = test_df.apply(lambda x: pd.to_datetime(x[6]),axis=1)
test_df["hearing_date"] = test_df.apply(lambda x: pd.to_datetime(x[7]),axis=1)
train_df["time_to_hearing"] = train_df["hearing_date"] - train_df["ticket_issued_date"]
test_df["time_to_hearing"] = test_df["hearing_date"] - test_df["ticket_issued_date"]

test_df["time_to_hearing"] = test_df["time_to_hearing"].map(str)
test_df["time_to_hearing"] = test_df["time_to_hearing"].replace('NaT',str(pd.Timestamp('1/1/18')-pd.Timestamp('1/1/18')))
test_df["time_to_hearing"] = test_df["time_to_hearing"].map(pd.Timedelta)

train_df["time_to_hearing"] = train_df["time_to_hearing"].apply(lambda x: x.days)
test_df["time_to_hearing"] = test_df["time_to_hearing"].apply(lambda x: x.days)

#Separating the 'ticket_id' column
ticket_id_train = train_df["ticket_id"]
ticket_id_test = test_df["ticket_id"]
train_df = train_df.drop(["ticket_id"],axis=1)
test_df = test_df.drop(["ticket_id"],axis=1)

### 4.3 Initial feature selection

In [5]:
#Removing the features that don't provide much useful information regarding the compliance - feature selection
#Also removing features that may directly relate to compliance and whereby cause data leakage
train_df = train_df.drop(["violation_zip_code","clean_up_cost","grafitti_status","discount_amount","non_us_str_code"],axis=1)
train_df = train_df.drop(["violator_name","balance_due","payment_status","payment_date","collection_status","compliance_detail"],axis=1)
train_df = train_df.drop(["admin_fee","state_fee","payment_amount","state","agency_name","inspector_name","country","violation_code"],axis=1)
train_df = train_df.drop(["zip_code","mailing_address_str_number"],axis=1)
test_df = test_df.drop(["violation_zip_code","clean_up_cost","grafitti_status","discount_amount","non_us_str_code"],axis=1)
test_df = test_df.drop(["violator_name","admin_fee","state_fee","state","agency_name","inspector_name","country","violation_code"],axis=1)
test_df = test_df.drop(["zip_code","mailing_address_str_number"],axis=1)

### 4.4 Label encoding

In [42]:
#Label encoding categorical variables
le = LabelEncoder()

le.fit(train_df["violation_street_name"].append(test_df["violation_street_name"]))
train_df["violation_street_name"] = le.transform(train_df["violation_street_name"])
test_df["violation_street_name"] = le.transform(test_df["violation_street_name"])

train_df["mailing_address_str_name"] = train_df["mailing_address_str_name"].map(str)
test_df["mailing_address_str_name"] = test_df["mailing_address_str_name"].map(str)
le.fit(train_df["mailing_address_str_name"].append(test_df["mailing_address_str_name"]))
train_df["mailing_address_str_name"] = le.transform(train_df["mailing_address_str_name"])
test_df["mailing_address_str_name"] = le.transform(test_df["mailing_address_str_name"])

le.fit(train_df["violation_description"].append(test_df["violation_description"]))
train_df["violation_description"] = le.transform(train_df["violation_description"])
test_df["violation_description"] = le.transform(test_df["violation_description"])

le.fit(train_df["disposition"].append(test_df["disposition"]))
train_df["disposition"] = le.transform(train_df["disposition"])
test_df["disposition"] = le.transform(test_df["disposition"])

### 4.5 Data preparation and training

In [ ]:
#Splitting the training dataset into features and target variables
y = train_df["compliance"]
x = train_df.drop(["compliance"],axis=1)

#Train/Test Split
X_train,X_test,y_train,y_test = train_test_split(x,y,random_state=0)

#Normalization using MinMax scaler
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

#for i in [0.1]:
#    for j in [25,30,35,40,45,50]:
#        for k in [5,6,7,8,9,10]:
        
#Fitting the model
#logreg = LogisticRegression(C=0.1,random_state=0)
#gbc = GradientBoostingClassifier(random_state=0,learning_rate=i,n_estimators=j,max_depth=k)
gbc = GradientBoostingClassifier(random_state=0,learning_rate=0.1,n_estimators=45,max_depth=5)
#model = logreg.fit(X_train,y_train)
#model = logreg.fit(X_train_scaled,y_train)
model = gbc.fit(X_train,y_train)

### 4.6 Prediction, evaluation and optimization

In [ ]:
#Determining the decision function
#y_score_lr = model.decision_function(X_test_scaled)
y_score_eval = model.decision_function(X_test)
y_proba_eval = model.predict_proba(X_test)
y_score = model.decision_function(test_df)
y_proba = model.predict_proba(test_df)

#print (y_score_lr[:20])
#print (y_proba[:20,1])
#print (y_proba.shape)
#print (ticket_id_test.shape)
#print (test_df.iloc[:20,:])
#print (y_test[:20])

#Determining ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_score_eval)
roc_auc = auc(fpr, tpr)

#Prediction
#y_predict = model.predict(X_test_scaled)
#y_predict_test_df = model.predict(test_df)
#y_predict = model.predict(X_test)

#Classification report
#print (classification_report(y_test,y_predict))

#print ("Learning rate: ",i)
#print ("Estimators: ",j)
#print ("Max depth: ",k)
#print ("")
#print ("Area under the curve (AUC) score ",roc_auc)
#print ("Accuracy ",accuracy_score(y_test,y_predict))
#print ("")
#print (test_df.columns)

#################################################### End of evaluation #########################################################

#grid_values = {'C': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
#grid_values = {'learning_rate':[0.001,0.01,0.1,1,10,100],'max_depth':[1,2,3,4,5],'n_estimators':[1,2,5,10]}

#grid_clf = GridSearchCV(gbc, param_grid = grid_values, scoring='roc_auc',cv=5)
#grid_clf.fit(X_train_scaled, y_train)
#grid_clf.fit(X_train, y_train)
#print (grid_clf.cv_results_)
#print ("")
#print ("Max AUC C parameter ",grid_clf.best_params_)
#print ("Best AUC score ",grid_clf.best_score_)

#y_decision_fn_scores = grid_clf.decision_function(X_test) 

################################################### End of Optimization ########################################################

#Creating the final compliance probability outcomes with ticket ID as index
compliance = pd.DataFrame()
compliance["ticket_id"] = ticket_id_test
compliance["compliance"] = y_proba[:,1]
compliance = compliance.set_index("ticket_id")

#plt.scatter(range(len(y_proba_lr[:20,1])),y_proba_lr[:20,1])
#plt.scatter(range(len(y_proba_lr[:20,0])),y_proba_lr[:20,0])
#plt.show()

print (compliance["compliance"])

In [11]:
'''
#Remove instances in "mailing_address_str_number" that start with "P.O. Bo"
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O. Bo',0)
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O. BO',0)
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O. Box',0)
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O. BOX',0)
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('PO BOX')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O.BOX')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P.O.')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('215-B')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('47-42')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('141-19')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('p.o. bo')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('98-8184')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('ONE')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('242-12')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P. O. B')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('94-28')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('P O BOX')
test_df["mailing_address_str_number"] = test_df["mailing_address_str_number"].replace('48-2')
'''


ticket_id
284932    0.084407
285362    0.010408
285361    0.043289
285338    0.041925
285346    0.058830
            ...   
376496    0.016640
376497    0.016640
376499    0.052997
376500    0.052242
369851    0.792380
Name: compliance, Length: 61001, dtype: float64
